In [1]:
import pandas as pd
import numpy as np
from random import gauss
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

def main():
    rPT = rSLm = np.linspace(0, 10, 21)
    count = 0
    for prod_ in product([10, 5, 0, -5, -10], [5, 10, 25, 50, 100]):
        count += 1
        coeffs = {'forecast': prod_[0], 'hl': prod_[1], 'sigma': 1}
        output = batch(coeffs, nIter=1e5, maxHP=100, rPT=rPT, rSLm=rSLm)
    return output

def batch(coeffs, nIter=1e5, maxHP=100, rPT=np.linspace(0.5, 10, 20), rSLm=np.linspace(0.5, 10, 20), seed=0):
    phi = 2 ** (-1.0 / coeffs['hl'])
    output1 = []
    for comb_ in product(rPT, rSLm):
        output2 = []
        for iter_ in range(int(nIter)):
            p, hp, count = seed, 0, 0
            while True:
                p = (1 - phi) * coeffs['forecast'] + phi * p + coeffs['sigma'] * gauss(0, 1)
                cP = p - seed
                hp += 1
                if cP > comb_[0] or cP < -comb_[1] or hp > maxHP:
                    output2.append(cP)
                    break
        mean, std = np.mean(output2), np.std(output2)
        print(comb_[0], comb_[1], mean, std, mean / std)
        output1.append((comb_[0], comb_[1], mean, std, mean / std))
    return output1


# 13.1.a

Suppose you are an execution trader. A client class you with an order to cover a short position she entered at a price of 100. She gives you two exit conditions: profit-taking at 90 and a stop-loss at 105.

Assuming the client belieces the price follows an O-U process, are these levels reasonable? 

**A: These levels appear to work well for Forecast values of -5 and all H-L values and Forecast values of -10 with H-L values between 25 and 100.**

# 13.1.b

Suppose you are an execution trader. A client class you with an order to cover a short position she entered at a price of 100. She gives you two exit conditions: profit-taking at 90 and a stop-loss at 105.

Can you think of an alternative stochastic process under which these levels make sense?

# 13.2.a

Fit the time series of dollar bars of E-mini S&P 500 futures to an O-U process. Given those parameters:

Produce a heat-map of Sharpe ratios for various profit-taking and stop-loss levels.


In [2]:
def gen_the_heat(forecast, hl, sigma=1):
    rPT = rSLm = np.linspace(0, 10, 21)
    coeffs = {'forecast': forecast, 'hl': hl, 'sigma': sigma}
    output = batch(coeffs, nIter=1e5, maxHP=100, rPT=rPT, rSLm=rSLm)
    return output

def pump_heatmap(coeffs, outputs):
    heatdf = pd.DataFrame(outputs)
    heatdfp = heatdf.pivot(1, 0, 4)
    plt.subplots() # give us a new figure
    return sns.heatmap(heatdfp.sort_index(ascending=False))

    

In [ ]:
from multiprocess import mpPandasObj
from synthetic_data import process_batch
runs = range(1,7)
rPT = rSLm = np.linspace(0, 10, 21)
coeffs_list = [{'forecast': x, 'hl': x, 'sigma': 1} for x in runs]

ret = mpPandasObj(process_batch, ('coeffs_list', coeffs_list), numThreads=6, linMols=True, nIter=1e4, maxHP=100, rPT=rPT, rSLm=rSLm)


In [ ]:
for coeffs, outputs in [x[0] for x in ret[1:]]:
    ax = pump_heatmap(coeffs, outputs)


# 13.2.b

Fit the time series of dollar bars of E-mini S&P 500 futures to an O-U process. Given those parameters:

What is the OTR?


**A: TODO: Figure out a good way of fitting to an O-U process.**


# 13.3.a

Repeat exercise 2, this time on a time series of dollar bars of

10-year U.S. Treasury Note futures


# 13.3.b

Repeat exercise 2, this time on a time series of dollar bars of

WTI Crude Oil futures


# 13.3.c

Repeat exercise 2, this time on a time series of dollar bars of

Are the results significantly different? Does this justify having execution traders specialized by product?


# 13.4.a

Repeat exercise 2 after splitting the time series into two parts:

The first time series ends on 3/15/2009.


# 13.4.b

Repeat exercise 2 after splitting the time series into two parts:

The second time series starts on 3/16/2009.


# 13.4.c

Repeat exercise 2 after splitting the time series into two parts:

Are the OTRs significantly different?


# 13.5

How long do you estimate it would take to derive OTRs on the 100 most liquid futures contracts worldwide? Considering the results from exercise 4, how often do you think you may have to re-calibrate the OTRs? Does it make sense to precompute this data?

# 13.6

Parallelize Snippets 13.1 and 13.2 using the `mpEngine` module described in Chapter 20.

**A: Code in synthetic_data.py**